In [19]:
import requests
import pandas as pd
import numpy as np

In [20]:
# data source : all from 公開資訊觀測站

# 營益分析彙總表
def FinState(marketType, year, season):
    """
    marketType : "sii"(上市) or "otc"(上櫃)
    year : for eample, 109
    season : for example, 3
    """
    url = 'https://mops.twse.com.tw/mops/web/ajax_t163sb06'
    r = requests.post(url, {
            'encodeURIComponent' : 1,
            'step' : 1,
            'firstin' : 1,
            'off' : 1,
            'TYPEK' : marketType,
            'year' : str(year),
            'season' : str(season),
        })
    r.encoding = 'utf8'
    df = pd.read_html(r.text, header=None)[0]
    df.columns = df.values[0]
    removeIndex = df[df["公司代號"] == "公司代號"].index.tolist()
    df.drop(index = removeIndex, axis = 0, inplace = True)
    df["年度"] = year
    df["季度"] = season
    return df

# 綜合損益彙總表
def IncState(marketType, year, season):
    """
    marketType : "sii"(上市) or "otc"(上櫃)
    year : for eample, 109
    season : for example, 3
    """
    url = 'https://mops.twse.com.tw/mops/web/ajax_t163sb04'
    r = requests.post(url, {
                'encodeURIComponent' : 1,
                'step' : 1,
                'firstin' : 1,
                'off' : 1,
                'TYPEK' : marketType,
                'year' : str(year),
                'season' : str(season),
            })
    r.encoding = 'utf8'
    
    if marketType == "sii":
        df = pd.read_html(r.text, header=None)[3]
        df = df[["公司代號","公司名稱","基本每股盈餘（元）"]]
        df["年度"] = year
        df["季度"] = season

        df1 = pd.read_html(r.text, header=None)[5]
        df1 = df1[["公司代號","公司名稱","基本每股盈餘（元）"]]
        df1["年度"] = year
        df1["季度"] = season
        return pd.concat([df, df1])
    else:
        df = pd.read_html(r.text, header=None)[2]
        df = df[["公司代號","公司名稱","基本每股盈餘（元）"]]
        df["年度"] = year
        df["季度"] = season
        return df

# 資產負債彙總表
def BSState(marketType, year, season):
    """
    marketType : "sii"(上市) or "otc"(上櫃)
    year : for eample, 109
    season : for example, 3
    """
    url = 'https://mops.twse.com.tw/mops/web/ajax_t163sb05' 
    r = requests.post(url, {
                'encodeURIComponent' : 1,
                'step' : 1,
                'firstin' : 1,
                'off' : 1,
                'TYPEK' : marketType,
                'year' : str(year),
                'season' : str(season),
            })
    r.encoding = 'utf8'
    
    if marketType == "sii":
        df = pd.read_html(r.text, header=None)[3]
        if year <= 106:
            df = df[["公司代號","公司名稱","資產總額","負債總額","權益總額","每股參考淨值"]]
            df.rename(columns = {"資產總額":"資產總計", "負債總額":"負債總計", "權益總額":"權益總計"} ,inplace = True)
        else:
            df = df[["公司代號","公司名稱","資產總計","負債總計","權益總計","每股參考淨值"]]
        df["年度"] = year
        df["季度"] = season

        df1 = pd.read_html(r.text, header=None)[5]
        if year <= 106:
            df1 = df1[["公司代號","公司名稱","資產總額","負債總額","權益總額","每股參考淨值"]]
            df1.rename(columns = {"資產總額":"資產總計", "負債總額":"負債總計", "權益總額":"權益總計"} ,inplace = True)
        else:
            df1 = df1[["公司代號","公司名稱","資產總計","負債總計","權益總計","每股參考淨值"]]
        df1["年度"] = year
        df1["季度"] = season
        return pd.concat([df, df1])
    else:
        df = pd.read_html(r.text, header=None)[2]
        if year <= 106:
            df = df[["公司代號","公司名稱","資產總額","負債總額","權益總額","每股參考淨值"]]
            df.rename(columns = {"資產總額":"資產總計", "負債總額":"負債總計", "權益總額":"權益總計"} ,inplace = True)
        else:
            df = df[["公司代號","公司名稱","資產總計","負債總計","權益總計","每股參考淨值"]]
        df["年度"] = year
        df["季度"] = season
        return df

In [21]:
""" make an example:
data range : from 106 s3 to 108 s3
You can change the parameter in year, season and the continue part, and the following code allows you to deal with any periods you like.
"""
marketType = ["sii", "otc"]
year = [106, 107, 108] # can change here
season = [1, 2, 3, 4]# can change here
FinStateList = []
IncStateList = []
BSStateList = []
for i in marketType:
    for j in year:
        for k in season:
            if ((j == 106) & (k < 3)) or ((j == 108) & (k > 3)):# can change here
                continue
            else:
                FinStateList.append(FinState(i, j, k))
                IncStateList.append(IncState(i, j, k))
                BSStateList.append(BSState(i, j, k))
                print("Done")
                print(i, j, k)

dfFinState = pd.concat(FinStateList, ignore_index=True, sort=False)
dfFinState = dfFinState.sort_values(by = ["公司代號","年度","季度"])
dfFinState.reset_index(drop = True, inplace = True)
problem1 = dfFinState[dfFinState["公司名稱"]=="櫃買測"].index.tolist()
problem2 = dfFinState[dfFinState["公司名稱"]=="測試帳號"].index.tolist()
dfFinState.drop(index = problem1 + problem2, axis = 0, inplace = True)
dfFinState.reset_index(drop = True, inplace = True)

dfIncState = pd.concat(IncStateList, ignore_index=True, sort=False)
dfIncState = dfIncState.sort_values(by = ["公司代號","年度","季度"])
dfIncState.reset_index(drop = True, inplace = True)
dfIncState["累計EPS"] = dfIncState["基本每股盈餘（元）"]
dfIncState = dfIncState[["公司代號","公司名稱","累計EPS","年度","季度"]]

dfBSState = pd.concat(BSStateList, ignore_index=True, sort=False)
dfBSState = dfBSState.sort_values(by = ["公司代號","年度","季度"])
dfBSState.reset_index(drop = True, inplace = True)

Done
sii 106 3
Done
sii 106 4
Done
sii 107 1
Done
sii 107 2
Done
sii 107 3
Done
sii 107 4
Done
sii 108 1
Done
sii 108 2
Done
sii 108 3
Done
otc 106 3
Done
otc 106 4
Done
otc 107 1
Done
otc 107 2
Done
otc 107 3
Done
otc 107 4
Done
otc 108 1
Done
otc 108 2
Done
otc 108 3


In [22]:
def FinReportCollation(dfFinState, dfIncState, dfBSState):
    #合併dfFinState dfIncState dfBSState，將所需資料合併至dfFinState
    dfFinState["累計EPS"] = dfIncState["累計EPS"]
    dfFinState['資產總計'] = dfBSState['資產總計']
    dfFinState['負債總計'] = dfBSState['負債總計']
    dfFinState['權益總計'] = dfBSState['權益總計']
    dfFinState['每股參考淨值'] = dfBSState['每股參考淨值']

    #將資料轉好格式，尤其是營益分析的資料都是累計的，需要跟累計EPS一樣自行轉換成單季，後續會做！
    dfFinState['營業收入(元)'] = [float(i)*1000000 for i in dfFinState['營業收入(百萬元)']] #轉成元為單位
    dfFinState["毛利率"] = [float(i)*0.01 for i in dfFinState['毛利率(%)(營業毛利)/(營業收入)']] #轉成實際小數點
    dfFinState['營業利益率'] = [float(i)*0.01 for i in dfFinState['營業利益率(%)(營業利益)/(營業收入)']]
    dfFinState['稅後純益率'] = [float(i)*0.01 for i in dfFinState['稅後純益率(%)(稅後純益)/(營業收入)']] #只轉稅後，因為稅前不打算用，在此略過。
    dfFinState['資產總計'] = [int(i)*1000 for i in dfFinState['資產總計']] #因為BS sheet那張資產負債權益都是以千元為單位，在此做修正。
    dfFinState['負債總計'] = [int(i)*1000 for i in dfFinState['負債總計']]
    dfFinState['權益總計'] = [int(i)*1000 for i in dfFinState['權益總計']]
    dfFinState['每股參考淨值'] = [float(i) for i in dfFinState['每股參考淨值']]
    dfFinState['累計EPS'] = [float(i) for i in dfFinState['累計EPS']]

    #為了要將累計EPS還有營益分析中的各年度按季累計上去的營業收入、毛利率、營業利益率、稅後純益率，拆解成單季，故在此做處理。
    SecurityList = sorted(set(list(dfFinState["公司代號"])))
    intervalDict = {}
    for key in dfFinState["公司代號"]:
        intervalDict[key] = intervalDict.get(key,0) + 1
    NotSuff = []
    for i in intervalDict:
        if intervalDict[i] < 9:
            NotSuff.append(i)
    #將不足9季的刪除

    #將NotSuff從SecurityList中移除，且要重新排序！
    SecurityList = sorted(list(set(SecurityList).difference(set(NotSuff))))

    #再將intervalDict做個修改
    for i in NotSuff:
        intervalDict.pop(i)

    #再把他們從dfFinState刪除
    NotsuffIndex = []
    for i in NotSuff:
        NotsuffIndex += dfFinState[dfFinState["公司代號"] == i].index.tolist()
    dfFinState.drop(index = NotsuffIndex, axis = 0, inplace = True)
    dfFinState.reset_index(drop = True, inplace = True)

    #首先將累計EPS、當年度按季累計的營業收入、毛利率、營業利益率、稅後純益率改成單季型態。
    EPS = []
    for i in range(len(SecurityList)):
        EPS.append("NA")#2017s3 無法計算單季
        for k in range(intervalDict[SecurityList[i]]-1):
            if i == 0:
                index = 1 + k
            else:
                index = sum(list(intervalDict.values())[0:i]) + 1 + k
            if k == 1: #2018s1的case
                EPSvalue = dfFinState['累計EPS'][index]
            elif k == 5: #2019s1的case
                EPSvalue = dfFinState['累計EPS'][index]
            else:
                EPSvalue = dfFinState['累計EPS'][index] - dfFinState['累計EPS'][index-1]
            EPS.append(EPSvalue)
    dfFinState["單季EPS"] = EPS

    OR = []
    for i in range(len(SecurityList)):
        OR.append("NA")#2017s3 無法計算單季
        for k in range(intervalDict[SecurityList[i]]-1):
            if i == 0:
                index = 1 + k
            else:
                index = sum(list(intervalDict.values())[0:i]) + 1 + k
            if k == 1: #2018s1的case
                ORvalue = dfFinState['營業收入(元)'][index]
            elif k == 5: #2019s1的case
                ORvalue = dfFinState['營業收入(元)'][index]
            else:
                ORvalue = dfFinState['營業收入(元)'][index] - dfFinState['營業收入(元)'][index-1]
            OR.append(ORvalue)
    dfFinState["單季營業收入"] = OR

    #毛利率比較特別，要先將那格乘上累積的營業收入，變成累積毛利後，再計算單季毛利，最後除以單季營收，即得單季毛利率了。
    dfFinState["累積毛利"] = dfFinState['毛利率'] * dfFinState['營業收入(元)']
    PM = []
    for i in range(len(SecurityList)):
        PM.append("NA")#2017s3 無法計算單季
        for k in range(intervalDict[SecurityList[i]]-1):
            if i == 0:
                index = 1 + k
            else:
                index = sum(list(intervalDict.values())[0:i]) + 1 + k
            if k == 1: #2018s1的case
                PMvalue = dfFinState['毛利率'][index]
            elif k == 5: #2019s1的case
                PMvalue = dfFinState['毛利率'][index]
            else:
                PMvalue = round(((dfFinState["累積毛利"][index] - dfFinState["累積毛利"][index-1])/dfFinState["單季營業收入"][index]), 4)
            PM.append(PMvalue)
    dfFinState["單季毛利率"] = PM

    #營業利益率比較特別，要先將那格乘上累積的營業收入，變成累積營業利益後，再計算單季營業利益，最後除以單季營收，即得單季營業利益率了。
    dfFinState["累積營業利益"] = dfFinState['營業利益率'] * dfFinState['營業收入(元)']
    OP = []
    for i in range(len(SecurityList)):
        OP.append("NA")#2017s3 無法計算單季
        for k in range(intervalDict[SecurityList[i]]-1):
            if i == 0:
                index = 1 + k
            else:
                index = sum(list(intervalDict.values())[0:i]) + 1 + k
            if k == 1: #2018s1的case
                OPvalue = dfFinState['營業利益率'][index]
            elif k == 5: #2019s1的case
                OPvalue = dfFinState['營業利益率'][index]
            else:
                OPvalue = round(((dfFinState["累積營業利益"][index] - dfFinState["累積營業利益"][index-1])/dfFinState["單季營業收入"][index]), 4)
            OP.append(OPvalue)
    dfFinState["單季營業利益率"] = OP

    #稅後純益率比較特別，要先將那格乘上累積的營業收入，變成累積稅後純益後，再計算單季稅後純益，最後除以單季營收，即得單季稅後純益率了。
    dfFinState["累積稅後純益"] = dfFinState['稅後純益率'] * dfFinState['營業收入(元)']
    ATNP = []
    for i in range(len(SecurityList)):
        ATNP.append("NA")#2017s3 無法計算單季
        for k in range(intervalDict[SecurityList[i]]-1):
            if i == 0:
                index = 1 + k
            else:
                index = sum(list(intervalDict.values())[0:i]) + 1 + k
            if k == 1: #2018s1的case
                ATNPvalue = dfFinState['稅後純益率'][index]
            elif k == 5: #2019s1的case
                ATNPvalue = dfFinState['稅後純益率'][index]
            else:
                ATNPvalue = round(((dfFinState["累積稅後純益"][index] - dfFinState["累積稅後純益"][index-1])/dfFinState["單季營業收入"][index]), 4)
            ATNP.append(ATNPvalue)
    dfFinState["單季稅後純益率"] = ATNP

    #ROE
    ROE = []
    for i in range(len(SecurityList)):
        ROE.append("NA")
        for k in range(intervalDict[SecurityList[i]]-1):
            if i == 0:
                index = 1 + k
            else:
                index = sum(list(intervalDict.values())[0:i]) + 1 + k
            ROEvalue = round(((dfFinState['單季稅後純益率'][index] * dfFinState['單季營業收入'][index])/((dfFinState['權益總計'][index-1] + dfFinState['權益總計'][index])/2)), 4)
            ROE.append(ROEvalue)
    dfFinState["ROE"] = ROE

    #負債佔資產比重
    dfFinState["負債佔資產比"] = round((dfFinState['負債總計']/dfFinState['資產總計']), 4)

    removeIndex = dfFinState[dfFinState['年度']==106][dfFinState['季度']==3].index.tolist()
    dfFinancialState = dfFinState.drop(index = removeIndex, axis = 0)
    dfFinancialState.reset_index(drop = True, inplace = True)
    #準備季增率/年增率，跟把A L E用百萬為單位表示，最後建檔。
    dfFinancialState['資產總額(百萬元)'] = round(dfFinancialState['資產總計']/1000000, 2)
    dfFinancialState['負債總額(百萬元)'] = round(dfFinancialState['負債總計']/1000000, 2)
    dfFinancialState['權益總額(百萬元)'] = round(dfFinancialState['權益總計']/1000000, 2)
    dfFinancialState["單季營收(百萬元)"] = dfFinancialState['單季營業收入']/1000000

    RevenueYoY=[]
    for i in range(len(set(list(dfFinancialState["公司代號"])))):
        for j in range(4):
            RevenueYoY.append("NA")
        for k in range(4):
            index = 8 * i + 4 + k
            RevenueYoYvalue = round(((dfFinancialState["單季營業收入"][index] - dfFinancialState["單季營業收入"][index-4])/abs(dfFinancialState["單季營業收入"][index-4])), 4)*100
            RevenueYoY.append(RevenueYoYvalue)
    dfFinancialState["營收年增率(%)"] = RevenueYoY

    PMQoQ=[]
    for i in range(len(set(list(dfFinancialState["公司代號"])))):
        PMQoQ.append("NA")
        for k in range(7):
            index = 8 * i + 1 + k
            PMQoQvalue = round(((dfFinancialState["單季毛利率"][index] - dfFinancialState["單季毛利率"][index-1])/abs(dfFinancialState["單季毛利率"][index-1])), 4)*100
            PMQoQ.append(PMQoQvalue)
    dfFinancialState["毛利率季增率(%)"] = PMQoQ

    EPSYoY=[]
    for i in range(len(set(list(dfFinancialState["公司代號"])))):
        for j in range(4):
            EPSYoY.append("NA")
        for k in range(4):
            index = 8 * i + 4 + k
            EPSYoYvalue = round(((dfFinancialState["單季EPS"][index] - dfFinancialState["單季EPS"][index-4])/abs(dfFinancialState["單季EPS"][index-4])), 4)*100
            EPSYoY.append(EPSYoYvalue)
    dfFinancialState["每股盈餘年增率(%)"] = EPSYoY

    dfFinancialState["ROE(%)"] = dfFinancialState["ROE"] * 100
    ROEYoY=[]
    for i in range(len(set(list(dfFinancialState["公司代號"])))):
        for j in range(4):
            ROEYoY.append("NA")
        for k in range(4):
            index = 8 * i + 4 + k
            ROEYoYvalue = round(((dfFinancialState["ROE(%)"][index] - dfFinancialState["ROE(%)"][index-4])/abs(dfFinancialState["ROE(%)"][index-4])), 4)*100
            ROEYoY.append(ROEYoYvalue)
    dfFinancialState["ROE年增率(%)"] = ROEYoY

    OPYoY=[]
    for i in range(len(set(list(dfFinancialState["公司代號"])))):
        for j in range(4):
            OPYoY.append("NA")
        for k in range(4):
            index = 8 * i + 4 + k
            OPYoYvalue = round(((dfFinancialState['單季營業利益率'][index]*dfFinancialState['單季營業收入'][index] - dfFinancialState['單季營業利益率'][index-4]*dfFinancialState['單季營業收入'][index-4])/abs(dfFinancialState['單季營業利益率'][index-4]*dfFinancialState['單季營業收入'][index-4])), 4)*100
            OPYoY.append(OPYoYvalue)
    dfFinancialState["營業利益年增率(%)"] = OPYoY

    ATNPYoY=[]
    for i in range(len(set(list(dfFinancialState["公司代號"])))):
        for j in range(4):
            ATNPYoY.append("NA")
        for k in range(4):
            index = 8 * i + 4 + k
            ATNPYoYvalue = round(((dfFinancialState['單季稅後純益率'][index]*dfFinancialState['單季營業收入'][index] - dfFinancialState['單季稅後純益率'][index-4]*dfFinancialState['單季營業收入'][index-4])/abs(dfFinancialState['單季稅後純益率'][index-4]*dfFinancialState['單季營業收入'][index-4])), 4)*100
            ATNPYoY.append(ATNPYoYvalue)
    dfFinancialState["稅後純益年增率(%)"] = ATNPYoY

    dfFinancialState['負債佔資產比(%)'] = dfFinancialState["負債佔資產比"] * 100
    dfFinancialState['單季毛利率(%)'] = dfFinancialState["單季毛利率"] * 100
    dfFinancialState['單季營業利益率(%)'] = dfFinancialState["單季營業利益率"] * 100
    dfFinancialState['單季稅後純益率(%)'] = dfFinancialState["單季稅後純益率"] * 100

    dfFinStateFinal = dfFinancialState[["公司代號","公司名稱","年度","季度","資產總額(百萬元)","負債總額(百萬元)","權益總額(百萬元)","每股參考淨值"
                      ,"單季EPS","單季營收(百萬元)","單季毛利率(%)","單季營業利益率(%)","單季稅後純益率(%)","營收年增率(%)","每股盈餘年增率(%)",
                     "ROE(%)","ROE年增率(%)","毛利率季增率(%)","營業利益年增率(%)","稅後純益年增率(%)","負債佔資產比(%)"]]
    dfFinStateFinal.rename(columns={"公司代號":"股票代號", "公司名稱":"股票名稱"}, inplace = True)
    return dfFinStateFinal.to_csv('財報資料庫1.csv', encoding="utf-8", index = 0)

In [23]:
FinReportCollation(dfFinState, dfIncState, dfBSState)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:95: RuntimeWarning: divide by zero encountered in double_scalars
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:95: RuntimeWarning: invalid value encountered in double_scalars
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:114: RuntimeWarning: divide by zero encountered in double_scalars
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:114: RuntimeWarning: invalid value encountered in double_scalars
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:133: RuntimeWarning: divide by zero encountered in double_scalars
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:133: RuntimeWarning: invalid value encountered in double_scalars
/Library/